In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from modules.stats import * 
from modules.PCA import * 
from modules.LDA import * 
from modules.read_data import * 
from modules.ROC import *

In [ ]:
path = '../../Assets/Synthetic Dataset'
X, Y, X_d, Y_d = read_SD(path)

In [ ]:
pca = False
lda = False
lda = ~pca & lda #if pca is true then lda is false

In [ ]:
if pca:
	l = 1
	train_mean, train_cov = stats(X)
	eigval, Q = PCA(train_cov, l)
	X = X @ Q
	X_d = X_d @ Q
if lda:
	pc = 1
	μ, Σ = stats(X)
	μk = []
	for c in range(2):
		μk.append(stats(X[Y == c])[0])
	μk = np.array(μk)
	v = LDA(X,Y,μ,μk,pc)
	X = np.abs(X @ v)
	X_d = np.abs(X_d @ v)

In [ ]:
classifier = Sequential()
classifier.add(Dense(units = 512, input_shape=(X.shape[1],), kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 512, input_shape=(X.shape[1],), kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.fit(X, Y, batch_size = 10, epochs = 100)

In [ ]:
score = classifier.predict(X_d)
y_pred = (score > 0.5)

In [ ]:
cm = confusion_matrix(Y_d, y_pred)
accuracy = accuracy_score(Y_d,y_pred)
ax = sns.heatmap(cm, annot = True, cmap ='plasma',linecolor ='black', linewidths = 1, fmt = '.0f', xticklabels = '12', yticklabels = '12')
ax.set_title(f'Accuracy = {accuracy}')

In [ ]:
TPR, FPR = ROC(Y_d, score)
plt.plot(FPR, TPR)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title("ROC")